In [ ]:
from secret_sauce import *
from ipywidgets import interact
%matplotlib inline

w = interact(show_color_threshold_results, red=red_widget, blue=blue_widget, green=green_widget)

In [ ]:
w = interact(show_edge_detection_results, low_threshold=low_widget, high_threshold=high_widget)

In [ ]:
# TODO(kristen): Add button to "rerun"
show_masking_results()

In [ ]:
show_regions_of_interest()

In [ ]:
show_hough_lines()